# Laminar Notebook Sample

 ## Include all the necessary imports

In [1]:
from dispel4py.base import ProducerPE, IterativePE, ConsumerPE
from dispel4py.workflow_graph import WorkflowGraph
import random
from client import d4pClient,Process

[*] Consider setting GitHub token to avoid hitting rate limits. 
For more info, see: https://docs.github.com/authentication/keeping-your-account-and-data-secure/creating-a-personal-access-token


Some weights of RobertaModel were not initialized from the model checkpoint at microsoft/reacc-py-retriever and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## Define your PEs

In [2]:
class NumberProducer(ProducerPE):
    def __init__(self):
        ProducerPE.__init__(self)
        
    def _process(self, inputs):
        # this PE produces one input
        result= random.randint(1, 1000)
        return result

class IsPrime(IterativePE):
    def __init__(self):
        IterativePE.__init__(self)
    def _process(self, num):
        # this PE consumes one input and produces one output
        print("before checking data - %s - is prime or not\n" % num, end="")
        if all(num % i != 0 for i in range(2, num)):
            return num

class PrintPrime(ConsumerPE):
    def __init__(self):
        ConsumerPE.__init__(self)
    def _process(self, num):
        # this PE consumes one input
        print("the num %s is prime\n" % num, end="")

## Define your Workflow

In [3]:
producer = NumberProducer()
isprime = IsPrime()
printprime = PrintPrime()

graph = WorkflowGraph()
graph.connect(producer, 'output', isprime, 'input')
graph.connect(isprime, 'output', printprime, 'input')

## Create user (if needed) and Login

In [4]:
client = d4pClient()

client.register("user_notebook","1234")

client.login("user_notebook","1234")

'user_notebook'

## Register the Workflow

In [21]:
client.register_Workflow(graph,"graph_sample", description="a workflow that produce random number and print only the prime ones.")

227

## Getting a Workflow

In [22]:
workflow = client.get_Workflow('graph_sample')
print("Workflow object %s" % workflow[0])

Workflow object <dispel4py.workflow_graph.WorkflowGraph object at 0x14832a980>


## Getting a PE

In [23]:
pe = client.get_PE('isprime')
print("PE object %s" % pe[0])

PE object <__main__.IsPrime object at 0x14833f670>


## Literal Search

In [24]:
print("\n Literal to Search on PEs \n")
results=client.search_Registry_Literal("prime","pe")
for r in results:
    print(r)


 Literal to Search on PEs 


REGISTRY

Result 1: ID: 228
PE Name: IsPrime
Description: This method checks the number of missing entries in the sequence.

Result 2: ID: 230
PE Name: PrintPrime
Description: Process the sequence of words in the sequence of words in the sequence of words in the sequence



## Semantic Search (text-to-code) on PEs

In [29]:
print("\n Semantic to Search on PEs \n")
results=client.search_Registry_Semantic("checks prime numbers","pe")
for r in results:
    print(r)


 Semantic to Search on PEs 

Performing similarity search on pe, with query type: text
Encoding query as text...
   peId          peName                                        description  \
0   228         IsPrime  This method checks the number of missing entri...   
1   229  NumberProducer  This function is called to generate a random s...   
2   230      PrintPrime  Process the sequence of words in the sequence ...   

   cosine_similarity  
0           0.322963  
1           0.215625  
2           0.102347  


In [30]:
print("The most similar PE is %s" %results[0])

The most similar PE is <__main__.IsPrime object at 0x14aebd120>


## Semantic Search (text-to-code) on Workflows

In [26]:
print("\n Semantic to Search on Workflows \n")
results=client.search_Registry_Semantic("checks prime numbers","workflow")
for r in results:
    print(r)


 Semantic to Search on Workflows 

Performing similarity search on workflow, with query type: text
Encoding query as text...
   workflowId   workflowName  \
0         227  WorkflowGraph   

                                         description  cosine_similarity  
0  a workflow that produce random number and prin...           0.536776  


In [28]:
print("The most similar workflow is %s" %results[0])

The most similar workflow is <dispel4py.workflow_graph.WorkflowGraph object at 0x14aebd2d0>


## Code Recommendation (code-to-code) on PEs

In [31]:
print("\n Text to Code Search on PEs\n")
results=client.code_Recommendation("random.randint(1, 1000)","pe")
for r in results:
    print(r)


 Text to Code Search on PEs


   peId          peName                                        description  \
0   229  NumberProducer  This function is called to generate a random s...   

   score                   simlarFunc  
0    8.0  def _process(self, inputs):  


In [32]:
print("The most similar PE is %s" %results[0])

The most similar PE is <__main__.NumberProducer object at 0x14aebd4e0>


## Code Recommendation (code-to-code) on Workflows

In [33]:
print("\n Text to Code Search on PEs\n")
results=client.code_Recommendation("random.randint(1, 1000)","workflow")
for r in results:
    print(r)


 Text to Code Search on PEs


   workflowId  workflowName  \
0         227  graph_sample   

                                         description  occurrences  
0  a workflow that produce random number and prin...            1  


In [34]:
print("The most similar workflow is %s" %results[0])

The most similar workflow is <dispel4py.workflow_graph.WorkflowGraph object at 0x14aebea10>


## Running the workflow sequentially - Simple Mapping

In [38]:
print("\n Running the Workflow Sequentially\n")
client.run(graph,input=40)


 Running the Workflow Sequentially

{'NumberProducer': 40}
before checking data - 946 - is prime or not
before checking data - 226 - is prime or not
before checking data - 667 - is prime or not
before checking data - 914 - is prime or not
before checking data - 997 - is prime or not
before checking data - 520 - is prime or not
before checking data - 696 - is prime or not
before checking data - 623 - is prime or not
before checking data - 458 - is prime or not
before checking data - 900 - is prime or not
before checking data - 574 - is prime or not
before checking data - 27 - is prime or not
before checking data - 907 - is prime or not
before checking data - 437 - is prime or not
before checking data - 798 - is prime or not
before checking data - 279 - is prime or not
before checking data - 834 - is prime or not
before checking data - 425 - is prime or not
before checking data - 198 - is prime or not
before checking data - 432 - is prime or not
before checking data - 525 - is prime or 

{}

## Running the Workflow in parallel - Multiprocessing maping

In [40]:
print("\n Running the Workflow in Parallel - Multi mapping\n")
client.run_multiprocess(graph,input=40)


 Running the Workflow in Parallel - Multi mapping

{'NumberProducer': 40}
Processes: {'NumberProducer0': range(0, 1), 'IsPrime1': range(1, 5), 'PrintPrime2': range(5, 9)}
NumberProducer0 (rank 0): Processed 40 iterations.
before checking data - 282 - is prime or not
before checking data - 673 - is prime or not
before checking data - 701 - is prime or not
before checking data - 921 - is prime or not
before checking data - 240 - is prime or not
before checking data - 873 - is prime or not
before checking data - 572 - is prime or not
before checking data - 297 - is prime or not
before checking data - 554 - is prime or not
before checking data - 306 - is prime or not
before checking data - 489 - is prime or not
before checking data - 308 - is prime or not
before checking data - 513 - is prime or not
before checking data - 3 - is prime or not
IsPrime1 (rank 1): Processed 10 iterations.
before checking data - 123 - is prime or not
before checking data - 803 - is prime or not
before checking

## Running the workflow Dynamically - Redis mapping

In [41]:
print("\n Running the Workflow Dynamically - Redis mapping\n")
client.run_dynamic(graph,input=40)


 Running the Workflow Dynamically - Redis mapping

{'NumberProducer': 40}
Starting 10 workers communicating
process:0 for instance:None redis connection created.
process:1 for instance:None redis connection created.
process:2 for instance:None redis connection created.
process:3 for instance:None redis connection created.
process:4 for instance:None redis connection created.
process:5 for instance:None redis connection created.
before checking data - 965 - is prime or not
process:6 for instance:None redis connection created.
before checking data - 353 - is prime or not
before checking data - 475 - is prime or not
before checking data - 179 - is prime or not
before checking data - 646 - is prime or not
before checking data - 767 - is prime or not
before checking data - 485 - is prime or not
before checking data - 821 - is prime or not
before checking data - 821 - is prime or not
before checking data - 723 - is prime or not
before checking data - 843 - is prime or not
before checking da

## Get all the elements from the registry

In [42]:
registry = client.get_Registry()
for elem in registry:
    print(elem)


REGISTRY

Result 1: ID: 228
PE Name: IsPrime
Description: This method checks the number of missing entries in the sequence.

Result 2: ID: 229
PE Name: NumberProducer
Description: This function is called to generate a random sequence of unique IDs.

Result 3: ID: 230
PE Name: PrintPrime
Description: Process the sequence of words in the sequence of words in the sequence of words in the sequence

Result 4: ID: 227
Workflow Name: graph_sample
Description: a workflow that produce random number and print only the prime ones.



## Describing a PE

In [45]:
data = client.get_PE("IsPrime")
isprime = data[0]
sc = data[1]
client.describe(isprime, sc)

PE name: IsPrime
inputconnections: {'input': {'name': 'input'}} 
outputconnections: {'output': {'name': 'output'}} 
numprocesses: 1 


## Update PE description - it needs the ID of the PE:

In [47]:
client.update_PE_Description(230, "updated PE description")
registry = client.get_Registry()
for elem in registry:
    print(elem)


REGISTRY

Result 1: ID: 228
PE Name: IsPrime
Description: This method checks the number of missing entries in the sequence.

Result 2: ID: 229
PE Name: NumberProducer
Description: This function is called to generate a random sequence of unique IDs.

Result 3: ID: 230
PE Name: PrintPrime
Description: updated PE description

Result 4: ID: 227
Workflow Name: graph_sample
Description: a workflow that produce random number and print only the prime ones.



## Update Workflow description - it needs the ID of the Workflow

In [48]:
client.update_Workflow_Description(227, "updated PE description")
registry = client.get_Registry()
for elem in registry:
    print(elem)


REGISTRY

Result 1: ID: 228
PE Name: IsPrime
Description: This method checks the number of missing entries in the sequence.

Result 2: ID: 229
PE Name: NumberProducer
Description: This function is called to generate a random sequence of unique IDs.

Result 3: ID: 230
PE Name: PrintPrime
Description: updated PE description

Result 4: ID: 227
Workflow Name: graph_sample
Description: updated PE description

